# Introduction
A neural network, sometimes also referred as multi layer perceptron, is basically an algorithm that is used to learn more sophisticated features than what a perceptron can learn. It does this by using multiple layers of perceptrons. The image below explains a little about the architecture it follows
<center> **Neural Network** </center>
![Neural Network architecture](NN_demo_image.jpeg "A Neural Network")

The inputs are given to the *input layer*, the hidden layers simply perform a weighed sum of the outputs of the previous layer, add it with a bias and then pass it through a function. Usually, the same activation function is used for every neuron in one layer. If you've gone through my perceptron notebook, you can perceive that every neuron is basically acting like a perceptron (hence the name _multi layer perceptron_)

In [ ]:
import numpy as np
import pickle
import os
from matplotlib import pyplot as plt

np.random.seed(2)